In [99]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# Load the CSV file
df = pd.read_csv('/content/Tag_dataset.csv')

# Data Cleaning
df.dropna(subset=['Description'], inplace=True)  # Drop rows with missing descriptions
df.drop_duplicates(subset=['Description'], keep='first', inplace=True)  # Remove duplicates

# Text Preprocessing with Lemmatization and Stopword Removal
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

df['lemmatized_description'] = df['Description'].apply(lemmatize_text)

# Remove Stopwords from Lemmatized Text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    return ' '.join(filtered_tokens)

df['cleaned_lemmatized_description'] = df['lemmatized_description'].apply(remove_stopwords)

# Create Array of Words from Cleaned Lemmatized Description
def create_word_array(text):
    return word_tokenize(text)

df['word_array'] = df['cleaned_lemmatized_description'].apply(create_word_array)

# Perform POS Tagging on Each Word in Each Array
def pos_tag_words(word_array):
    return pos_tag(word_array)

df['pos_tagged_words'] = df['word_array'].apply(pos_tag_words)

# Create Separate Columns for Specific POS Tags
def extract_pos_tags(tagged_words, pos_tag):
    return ' '.join([word for word, tag in tagged_words if tag == pos_tag])

df['Nouns'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'NN') + ' ' + extract_pos_tags(x, 'NNS') + ' ' + extract_pos_tags(x, 'NNP') + ' ' + extract_pos_tags(x, 'NNPS'))
df['Verbs'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'VB') + ' ' + extract_pos_tags(x, 'VBD') + ' ' + extract_pos_tags(x, 'VBG') + ' ' + extract_pos_tags(x, 'VBN') + ' ' + extract_pos_tags(x, 'VBP') + ' ' + extract_pos_tags(x, 'VBZ'))
df['Adjectives'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'JJ'))
# Save the preprocessed data with specific POS tagged words to a new CSV file
df.to_csv('Final_pd.csv', index=False)

In [100]:
import tensorflow as tf
from tensorflow import keras

In [101]:
df.head()

,ID,Name,Description,Tags,lemmatized_description,cleaned_lemmatized_description,word_array,pos_tagged_words,Nouns,Verbs,Adjectives
0,1.0,Sarah,"Sarah is a dedicated student with a passion for Mathematics and Physics. Her interests lie in the realms of Astrophysics and Quantum Mechanics, where she delves into the mysteries of the universe ...",Physics,"Sarah is a dedicated student with a passion for Mathematics and Physics . Her interest lie in the realm of Astrophysics and Quantum Mechanics , where she delf into the mystery of the universe with...","Sarah dedicated student passion Mathematics Physics . interest lie realm Astrophysics Quantum Mechanics , delf mystery universe fervor curiosity","[Sarah, dedicated, student, passion, Mathematics, Physics, ., interest, lie, realm, Astrophysics, Quantum, Mechanics, ,, delf, mystery, universe, fervor, curiosity]","[(Sarah, NNP), (dedicated, VBD), (student, NN), (passion, NN), (Mathematics, NNPS), (Physics, NNPS), (., .), (interest, NN), (lie, NN), (realm, NN), (Astrophysics, NNP), (Quantum, NNP), (Mechanics...",student passion interest lie realm fervor curiosity Sarah Astrophysics Quantum Mechanics Mathematics Physics,dedicated mystery,universe
1,2.0,Alex,"Alex is a tech enthusiast studying Computer Science and Data Science. His interests revolve around Artificial Intelligence and Machine Learning, where he explores the cutting-edge technologies sha...",Computer Science,"Alex is a tech enthusiast studying Computer Science and Data Science . His interest revolve around Artificial Intelligence and Machine Learning , where he explores the cutting-edge technology shap...","Alex tech enthusiast studying Computer Science Data Science . interest revolve around Artificial Intelligence Machine Learning , explores cutting-edge technology shaping future computing .","[Alex, tech, enthusiast, studying, Computer, Science, Data, Science, ., interest, revolve, around, Artificial, Intelligence, Machine, Learning, ,, explores, cutting-edge, technology, shaping, futu...","[(Alex, NNP), (tech, VBD), (enthusiast, RB), (studying, VBG), (Computer, NNP), (Science, NNP), (Data, NNP), (Science, NNP), (., .), (interest, NN), (revolve, NN), (around, IN), (Artificial, NNP), ...",interest revolve technology computing Alex Computer Science Data Science Artificial Intelligence Machine Learning,tech studying shaping explores,cutting-edge future
2,3.0,Emily,"Emily is an avid reader and historian, focusing on Literature and History. She finds joy in Creative Writing and delves deep into the narratives of Ancient Civilizations, drawing inspiration from ...",Literature,"Emily is an avid reader and historian , focusing on Literature and History . She find joy in Creative Writing and delf deep into the narrative of Ancient Civilizations , drawing inspiration from t...","Emily avid reader historian , focusing Literature History . find joy Creative Writing delf deep narrative Ancient Civilizations , drawing inspiration past creative endeavor .","[Emily, avid, reader, historian, ,, focusing, Literature, History, ., find, joy, Creative, Writing, delf, deep, narrative, Ancient, Civilizations, ,, drawing, inspiration, past, creative, endeavor...","[(Emily, RB), (avid, JJ), (reader, NN), (historian, NN), (,, ,), (focusing, VBG), (Literature, NNP), (History, NNP), (., .), (find, VB), (joy, JJ), (Creative, NNP), (Writing, NNP), (delf, PRP), (d...",reader historian inspiration endeavor Literature History Creative Writing Ancient Civilizations,find focusing drawing,avid joy deep narrative creative
3,4.0,James,"James is a science enthusiast excelling in Biology and Chemistry. His interests in Genetics and Biochemistry drive his research and experiments, as he seeks to unravel the complexities of life at ...",Chemistry,"James is a science enthusiast excelling in Biology and Chemistry . His interest in Genetics and Biochemistry drive his research and experiment , a he seek to unravel the complexity of life at a mo...","James science e

In [102]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_lemmatized_description'])
# check unique words count
len(tokenizer.word_index)

# check unique words count
vocab_size = len(tokenizer.word_index) + 1
vocab_size

sequences = tokenizer.texts_to_sequences(df['cleaned_lemmatized_description'])
i = 0
print(df['cleaned_lemmatized_description'][i], '\n'), print(sequences[i])

Sarah dedicated student passion Mathematics Physics . interest lie realm Astrophysics Quantum Mechanics , delf mystery universe fervor curiosity 

[141, 40, 8, 6, 142, 71, 1, 41, 24, 143, 144, 145, 7, 72, 73, 146, 147]


(None, None)

In [103]:
seq_lengths = []

for i in sequences:
    seq_lengths.append(len(i))
print("30th percentile: ", pd.Series(seq_lengths).quantile(0.3))
print("40th percentile: ", pd.Series(seq_lengths).quantile(0.4))
print("50th percentile: ", pd.Series(seq_lengths).quantile(0.5))
print("60th percentile: ", pd.Series(seq_lengths).quantile(0.6))
print("70th percentile: ", pd.Series(seq_lengths).quantile(0.7))
print("80th percentile: ", pd.Series(seq_lengths).quantile(0.8))
print("90th percentile: ", pd.Series(seq_lengths).quantile(0.9))
print("95th percentile: ", pd.Series(seq_lengths).quantile(0.95))
print("99th percentile: ", pd.Series(seq_lengths).quantile(0.99))

30th percentile:  21.0
40th percentile:  22.0
50th percentile:  22.5
60th percentile:  23.0
70th percentile:  24.299999999999997
80th percentile:  25.0
90th percentile:  25.0
95th percentile:  25.549999999999997
99th percentile:  26.0


In [104]:
max_length = 22

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df['Tags'])
y = multilabel_binarizer.transform(df['Tags'])
padded_seq.shape, y.shape

((30, 22), (30, 37))

In [105]:
pip install tensorflow

In [106]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define a simple Keras model
def create_model():
    model = Sequential()
    model.add(Dense(10, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create a KerasClassifier using scikeras
keras_model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)

# Use the KerasClassifier in your scikit-learn workflow
# For example, you can use it in a scikit-learn Pipeline or GridSearchCV

In [112]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y,
                                                    test_size=0.1,
                                                    random_state=9)
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [119]:

model = Sequential()
model.add(Embedding(vocab_size +1, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 5, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation = "sigmoid"))
#model.add(Activation('sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()
tf.keras.layers.Flatten()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 22, 128)           45440     
                                                                 
 dropout_3 (Dropout)         (None, 22, 128)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 18, 300)           192300    
                                                                 
 global_max_pooling1d_3 (Gl  (None, 300)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_3 (Dense)             (None, 100)               30100     
                                                                 
Total params: 267840 (1.02 MB)
Trainable params: 267840 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [120]:
callbacks = [
             EarlyStopping(patience=3),
             ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)
            ]


In [121]:
history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=64,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/15


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [116]:
preds = model.predict(x_val)

1/1 [==============================] - 0s 330ms/step
